In [1]:
#Import relevant librariess
#Importing packages
from datetime import date
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import pymysql
from sqlalchemy import create_engine, text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR')

'fr_FR'

In [2]:
#Dates for the most recent period
from datetime import date
#today = date.today()
start_date = pd.to_datetime('2024-10-01')
end_date = pd.to_datetime('2024-12-31')
#Dates for the period before the most recent period
start_date_last = pd.to_datetime('2025-01-01')
end_date_last = pd.to_datetime(date.today())

#=================================
start_year = pd.to_datetime('2025-10-01')
end_year =pd.to_datetime ('2025-12-31')

In [3]:
# Charger les variables d'environnement
load_dotenv('dot.env')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
db = os.getenv('MYSQL_DB')

# Connexion à la base de données
conn_text = f'mysql+pymysql://{user}:{password}@{host}/{db}'
engine = create_engine(conn_text)

# Lire la requête SQL à partir du fichier
with open('./Mastersheet PTME.sql', 'r') as file: 
    sql_query = file.read().replace('use caris_db;', '')
    
# Exécuter la requête SQL et stocker les résultats dans un DataFrame
cv_df1 = pd.read_sql_query(sql_query, engine)

# Afficher les premières lignes du DataFrame
display(cv_df1.head(2))
display(cv_df1.shape)

# Fermer la connexion
engine.dispose()


True

,office,network,departement,commune,section,hospital_name,id_patient,mother_patient_code,created_at,site,...,Infant_Prophylaxis_Commcare,woman_does_not_belong_to_a_club_definition,delivery_date_merge,delivery_probality_date_merge,DPA_calculated,allaitante,enceinte,enceinte_bt_7_14,last_followup_date,next_appointment_date
0,BOM,UGP,Nord-Ouest,Bombardopolis,1ère Plate Forme,Hôpital Evangelique de Bombardopolis,187733,BOM/HEBA/ST03800,2024-02-26 21:48:09,BOM/HEBA,...,None,None,2023-11-04,2023-11-04,2023-11-05,yes,no,no,2023-10-24,0000-00-00
1,BOM,UGP,Nord-Ouest,Bombardopolis,1ère Plate Forme,Hôpital Evangelique de Bombardopolis,189403,BOM/HEBA/ST03834,2024-04-09 15:37:24,BOM/HEBA,...,None,Partenaire nâ€™est pas au courant de son statut,2024-02-18,2024-02-18,2024-02-20,yes,no,no,None,None


(2993, 87)

In [4]:
cv_df1.rename(columns={'mother_patient_code':'patient_code'}, inplace=True)

In [5]:
# Define the date range variables
start_date = '2024-10-01'
end_date = '2025-03-13'
p_start = '2024-01-01'
p_end = '2025-03-13'

# Define the SQL query
sql_query = f"""
SELECT a.*,
p.patient_code, 
tmi.telephone as telephone1 , 
tmi.telephone2 as Telephone2
FROM 
((SELECT
        tmi.id_patient
    FROM
        tracking_pregnancy tp
    LEFT JOIN tracking_motherbasicinfo tmi ON tmi.id_patient = tp.id_patient_mother
    WHERE
        (tp.dpa BETWEEN '{start_date}' AND '{end_date}')
            OR ((tp.ddr + INTERVAL 9 MONTH + INTERVAL 7 DAY) BETWEEN '{start_date}' AND '{end_date}')
            OR (tp.actual_delivery_date BETWEEN '{start_date}' AND '{end_date}') OR (tp.created_at between '{p_start}' and '{p_end}') OR (tmi.created_at between '{p_start}' and '{p_end}'))
            UNION (SELECT
        p.id AS id_patient
    FROM
        testing_mereenfant tme
    LEFT JOIN patient p ON p.patient_code = CONCAT(tme.mother_city_code, '/', tme.mother_hospital_code, '/', tme.mother_code)
    WHERE
        tme.infant_dob BETWEEN '{start_date}' AND '{end_date}') UNION (SELECT
        id_patient_mother AS id_patient
    FROM
        tracking_children tc
        LEFT JOIN tracking_motherbasicinfo tmi2 on tmi2.id_patient = tc.id_patient_mother
    LEFT JOIN testing_mereenfant tm ON tm.id_patient = tc.id_patient_child
    WHERE (tmi2.id is not null) AND
        (tm.infant_dob BETWEEN '{start_date}' AND '{end_date}')
            OR (tc.dob BETWEEN '{start_date}' AND '{end_date}'))) a
            left join tracking_motherbasicinfo tmi on a.id_patient=tmi.id_patient
            left join patient p on a.id_patient = p.id
"""

# Execute the query and store the result in a DataFrame
tel = pd.read_sql_query(sql_query, engine)
# Remplace les valeurs NaN par '---'
tel['telephone1'] = tel['telephone1'].fillna('---')
tel['Telephone2'] = tel['Telephone2'].fillna('---')

# Remplace les valeurs vides par '---'
tel['telephone1'] = tel['telephone1'].replace('', '---')
tel['Telephone2'] = tel['Telephone2'].replace('', '---')

# Supprime les doublons en gardant la première occurrence pour chaque patient_code
tel = tel.drop_duplicates(subset=['patient_code'], keep='first')
# Display the first few rows of the DataFrame
display(tel.head())
display(tel.shape[0])

,id_patient,patient_code,telephone1,Telephone2
0,30319.0,STM/CSPP/ST01035,32917023,32287091
1,34492.0,PDP/CMEB/ST00862,41412923,---
2,40299.0,PDP/CMEB/ST04424,42914837,---
3,47941.0,LIM/HOSJ/ST01951,---,---
4,37577.0,PIL/HOEP/ST00421,---,---


1716

In [6]:
# Lire la requête SQL à partir du fichier
with open('./All_sessions_clubs_PTME.sql', 'r') as file: 
    sql_query = file.read().replace('USE caris_db;', '')

# Exécuter la requête SQL et stocker les résultats dans un DataFrame
club_info = pd.read_sql_query(sql_query, engine)

# Afficher les premières lignes du DataFrame
club_info = club_info[club_info['has_session_siy']=="yes"]
display(club_info.shape[0])
display(club_info.head(2))



2178

,departement,commune,section,club_name,hospital,patient_code,first_name,last_name,dob,first_session_date,last_session_date,days_from_last_session,has_session_siy
1,Nord,Milot,2ème Bonnet àl'Evêque,Fanm Vanyan 6,Hôpital Sacre Coeur de Milot,CAP/HSCM/ST01004,Phadette,Gracius,1998-01-01,2022-08-23,2025-01-21,54,yes
2,Nord,Milot,2ème Bonnet àl'Evêque,Fanm Vanyan5,Hôpital Sacre Coeur de Milot,CAP/HSCM/ST00913,Ilianie,Angrand,1997-09-01,2021-03-09,2025-01-21,54,yes


In [7]:
# Fusionner club_infos et tel sur la colonne 'patient_code' (équivalent de left_join en R)
club_info = club_info.merge(tel, on='patient_code', how='left', suffixes=('_club', '_tel'))

# Remplacer les valeurs NaN par '---' pour les colonnes 'telephone1' et 'Telephone2'
club_info['telephone1'] = club_info['telephone1'].fillna('---')
club_info['Telephone2'] = club_info['Telephone2'].fillna('---')
display(club_info.shape[0])
display(club_info.head(2))

2178

,departement,commune,section,club_name,hospital,patient_code,first_name,last_name,dob,first_session_date,last_session_date,days_from_last_session,has_session_siy,id_patient,telephone1,Telephone2
0,Nord,Milot,2ème Bonnet àl'Evêque,Fanm Vanyan 6,Hôpital Sacre Coeur de Milot,CAP/HSCM/ST01004,Phadette,Gracius,1998-01-01,2022-08-23,2025-01-21,54,yes,NaN,---,---
1,Nord,Milot,2ème Bonnet àl'Evêque,Fanm Vanyan5,Hôpital Sacre Coeur de Milot,CAP/HSCM/ST00913,Ilianie,Angrand,1997-09-01,2021-03-09,2025-01-21,54,yes,5494.0,---,---


In [ ]:
from ptme_fonction import download_files
# Appel de la fonction
download_files()

An error occurred while downloading from https://www.commcarehq.org/a/caris-test/data/export/custom/new/form/download/c1a3280f5e34a2b6078439f9b59ad72c/: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF724F1FE45+26629]
	(No symbol) [0x00007FF724E86010]
	(No symbol) [0x00007FF724D1931A]
	(No symbol) [0x00007FF724CF1753]
	(No symbol) [0x00007FF724D9F6BE]
	(No symbol) [0x00007FF724DBFA55]
	(No symbol) [0x00007FF724D97883]
	(No symbol) [0x00007FF724D60550]
	(No symbol) [0x00007FF724D61803]
	GetHandleVerifier [0x00007FF7252772BD+3529853]
	GetHandleVerifier [0x00007FF72528DA22+3621858]
	GetHandleVerifier [0x00007FF7252824D3+3575443]
	GetHandleVerifier [0x00007FF724FEB77A+860474]
	(No symbol) [0x00007FF724E9088F]
	(No symbol) [0x00007FF724E8CBC4]
	(No symbol) [0x00007FF724E8CD66]
	(No symbol) [0x00007FF724E7C2C9]
	BaseThreadInitThunk [0x00007FF9B592E8D7+23]
	RtlUserThrea

In [ ]:
# new calls file
#today_date=date.today()
appel = pd.read_excel(f"~/Downloads/Caris Health Agent - Femme PMTE  - APPELS PTME (created 2025-02-13) 2025-03-13.xlsx")

In [ ]:
appel.rename(columns={'form.APPELS_PTME.patient_code':'patient_code',
                   'form.APPELS_PTME.date_appel':'date',
                   'form.APPELS_PTME.is_ptme_available':'Trouvé'}, inplace=True)

In [ ]:
import pandas as pd
import re
# Function to assign commune names using regex
def assign_commune(name):
    if re.match(r'^1', name):
        return 'Cap-Haïtien'
    elif re.match(r'^2', name):
        return 'Port-au-Prince'
    elif re.match(r'^5', name):
        return 'Port-de-Paix'
    elif re.match(r'^6', name):
        return 'Gonaïves'
    else:
        return 'Autre'

# Function to filter rows by date, data column, and set commune and name values
def dataframe_for_period(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column])
    # Filter by date range
    df = df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]

    # Assign 'commune' based on 'username'
    df['commune'] = df['username'].apply(assign_commune)

    # Filter out 'Autre'
    df = df[df['commune'] != 'Autre']

    return df


In [ ]:
appel['date'].replace('---', '1901-01-01', inplace=True)

In [ ]:
appel = dataframe_for_period(appel,'date')
appel.shape

(4639, 39)

In [ ]:
appel['date'] = pd.to_datetime(appel['date'], errors='coerce')
appel['Trouvé'] = pd.to_numeric(appel['Trouvé'], errors='coerce')

appel_qi = appel[(appel['date'] >= start_date_last) & (appel['date'] <= end_date_last) &
                 (appel['Trouvé'] == 1)][['patient_code', 'date', 'Trouvé']].drop_duplicates('patient_code')
appel_qj = appel[(appel['date'] >= start_date) & (appel['date'] <= end_date) &
                 (appel['Trouvé'] == 1)][['patient_code', 'date', 'Trouvé']].drop_duplicates('patient_code')

In [ ]:
print(f"Nombre d'appels Qi: {appel_qi.shape[0]}")
print(f"Nombre d'appels Qj: {appel_qj.shape[0]}")

Nombre d'appels Qi: 1186
Nombre d'appels Qj: 736


In [ ]:
# create new columns to receive calls information
cv_df1['odk_call_q1'] = "no"
cv_df1["odk_call_q2"] = "no"

# make changes in services according to calls received by beneficiaries
for code in appel_qi['patient_code']:
    cv_df1.loc[(cv_df1['patient_code'] == code), 'odk_call_q1'] = 'yes'

for code in appel_qj['patient_code']:
    cv_df1.loc[(cv_df1['patient_code'] == code), 'odk_call_q2'] = 'yes'

cv_df1.loc[(cv_df1['odk_call_q1'] == 'yes'), 'q1'] = 'yes'
cv_df1.loc[(cv_df1['odk_call_q2'] == 'yes'), 'q2'] = 'yes'

In [ ]:
# Définir les plages de dates
start_qi, end_qi = pd.to_datetime('2024-10-01'), pd.to_datetime('2024-12-31')
start_qj, end_qj = pd.to_datetime('2025-01-01'), pd.to_datetime('2025-03-10')

# Convertir la colonne 'last_session_date_result' en datetime pour éviter les erreurs
club_info['last_session_date'] = pd.to_datetime(club_info['last_session_date'], errors='coerce')

# Créer les nouvelles colonnes 'club_qi' et 'club_qj' avec "Yes" ou "No"
club_info['club_q1'] = np.where(
    (club_info['last_session_date'] >= start_qi) & (club_info['last_session_date'] <= end_qi),
    "yes", "no"
)

club_info['club_q2'] = np.where(
    (club_info['last_session_date'] >= start_qj) & (club_info['last_session_date'] <= end_qj),
    "yes", "no"
)

In [ ]:
#served women in Q1 and Q2
q1q2 = cv_df1[(cv_df1.q1 == 'yes') & (cv_df1.q2 == 'yes')][['patient_code', 'club_q1', 'club_q2', 'on_arv', 'commune', 'age']]

#served women created while Q2
q2 = cv_df1[(cv_df1.patient_created_date >= start_date) & (cv_df1.patient_created_date <= end_date)][['patient_code', 'club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#Concatenation of the two previous datasets
OVC = pd.concat([q1q2, q2]).drop_duplicates('patient_code').reset_index(drop = True)

KeyError: "['club_q1', 'club_q2'] not in index"